### ARTIST 동의어 사전 만들기

* elastic Search의 가수를 매핑 시키기 위한 사용자 정의 가수의 사전을 만듭니다.

* 데이터 출처 : https://www.kaggle.com/datasets/kimjihoo/kpopdb
    * 가수 정보 : 1992 ~ 2020

In [13]:
"""
kaggle dataset download
- ~/.kaggle 파일에 kaggle.json 으로 token 발급 필요
"""

!kaggle datasets download -d kimjihoo/kpopdb

100%|█████████████████████████████████████████| 159k/159k [00:00<00:00, 677kB/s]
100%|█████████████████████████████████████████| 159k/159k [00:00<00:00, 676kB/s]


In [36]:
import os
import zipfile

zip_file_dir = os.path.join(os.getcwd(), "kpopdb.zip")
os.makedirs("data", exist_ok = True)
unzip_dir = "data"

with zipfile.ZipFile(zip_file_dir, "r") as zip_ref :
    zip_ref.extractall(unzip_dir)

In [53]:
"""
압축 푼 kpop 가수 정보 csv 파일을 불러온다.
"""
current_directory = os.getcwd()
folder_path = os.path.join(current_directory, "data")
file_path = os.listdir(folder_path)
file_paths = [file for file in os.listdir(folder_path) if file != "kpop_music_videos.csv"]
print(file_paths)

['kpop_idols_boy_groups.csv', 'kpop_idols_girl_groups.csv', 'kpop_idols.csv']


In [ ]:
"""
문자열 처리를 위한, 함수
"""
import re

# 공백 및 특수 문자 제거
def clean_text(text : str) -> str:
    clean_text = re.sub(r"\W+", "", text) 
    return clean_text.lower() if text.isalpha() else clean_text

# 문자열 조합 생성
def make_artist_synonyms(row : list) -> list :
    original_names = [str(original_name).lower() for original_name in row]
    cleaned_names = [clean_text(str(original_name)).lower() for original_name in row]
    return list(set(original_names + cleaned_names))

In [77]:
"""
kpop_idols_boy_groups.csv
kpop_idols_girl_groups.csv

두 파일만 먼저 동의어 제작
"""
import pandas as pd
dataframes = [pd.read_csv(folder_path + "/" + file, usecols = ['Name', 'Korean Name']) for file in file_paths if file != "kpop_idols.csv"]

# 모든 데이터 프레임을 하나로 병합
df_concatenated = pd.concat(dataframes, ignore_index=True)

group_synonyms_list = []
for index, row in df_concatenated.iterrows() :
    group_synonyms_list.append(make_artist_synonyms(row.tolist()))


In [78]:
"""
솔로 가수들만 동의어 제작
"""
import pandas as pd
df = pd.read_csv(folder_path + "/" + "kpop_idols.csv", usecols = ["Stage Name","Full Name","Korean Name","K. Stage Name"])

solo_synonyms_list = []
for index, row in df.iterrows() :
    solo_synonyms_list.append(make_artist_synonyms(row.tolist()))

In [81]:
"""
솔로 가수의 동의어 사전과 그룹 가수의 동의어 사전을 결합한다.
동의어 사전을 제작한다.
"""
from dotenv import load_dotenv
total_artist_synonyms_list = group_synonyms_list + solo_synonyms_list


load_dotenv()
ELASTIC_SYNONYMS_DIR = os.environ.get("ELASTIC_SYNONYMS_DIR")
os.makedirs(ELASTIC_SYNONYMS_DIR, exist_ok=True)
with open(ELASTIC_SYNONYMS_DIR + "/artist_synonyms.txt", "w", encoding = "utf-8") as file:
    for row in total_artist_synonyms_list :
        line = ', '.join(map(str, row))
        file.write(line + "\n")
print("가수 파일 저장 완료")


파일 저장 완료


In [82]:
"""
작업 완료 후, 데이터 자료 삭제
"""
import os, glob, shutil

zip_files = glob.glob(current_directory + "/*.zip")
csv_files = glob.glob(current_directory + "/*.csv")
for file_path in zip_files + csv_files :
    os.remove(file_path)
shutil.rmtree(current_directory + "/data")
